In [73]:
%run '01-02_Cartan_Group.ipynb'

class Parabolic_Subgroup_In_Irreducible_Cartan_Group ( object ) :

    def __eq__ ( self , other ) :
        """Tests if two objects of the class ``Parabolic_Subgroup_In_Irreducible_Cartan_Group`` coincide."""
        return type( other ) == self.__class__ and other.__repr__() == self.__repr__()


    def __init__( self , Parent_Group , Included_Nodes ) :
        """
        Initialize an algebraic group of given cartan type over a base field.

        INPUT:
        - ``Parent_Group`` -- Cartan_Group; the parent group G
        - ``Included_Nodes`` -- set; the set of included nodes. A parabolic subgroup is uniquely determined by
          taking the Borel subgroup B ⊂ G and adding the negative roots which are marked.

        OUTPUT: None.
        """
        # Test if the input variable ``Parent_Group`` is of the desired type.
        assert type( Parent_Group ) == Irreducible_Cartan_Group , 'The parent group must be an irreducible Cartan group.'
        self._Parent_Group = Parent_Group
        # Test if ``Included_Nodes`` is a set
        assert type( Included_Nodes ) == set , 'The amount of marked points must be given as set.'
        # Test if ``Included_Nodes`` is a subset of all nodes of G
        Nodes = set( Parent_Group.Cartan_Type().index_set() )
        assert Included_Nodes.issubset( Nodes ) , 'The set of marked nodes must be a subset of all nodes ' + str(Nodes) + '.'
        self._Included_Nodes = Included_Nodes


    def __ne__ ( self , other ) :
        """Tests if two objects of the class ``Parabolic_Subgroup_In_Irreducible_Cartan_Group`` do not coincide."""
        return not self == other


    def __repr__ ( self ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Parent_Group() , self.Included_Nodes()


    def __str__ ( self ) :
        """Returns a one-line string as short description."""
        return 'Parabolic subgroup P(' + str( self.Included_Nodes() ) + ') in Cartan group of type ' + self.Parent_Group().Cartan_String() + '.'


    def Cartan_String ( self ) :
        """Returns the Cartan string of the parabolic subgroup P."""
        Cartan_Family = self.Parent_Group().Cartan_Family()
        Cartan_Degree = self.Parent_Group().Cartan_Degree()
        Excluded_Nodes = self.Excluded_Nodes()

        Irreducible_Cartan_Strings = []
        for Connected_Component in self.Connected_Components_Of_Included_Nodes() :
            Cartan_Subtype = self.Parent_Group().Cartan_Type().subtype( Connected_Component )
            Irreducible_Cartan_Strings += [ Cartan_Subtype.type() + str(len(Cartan_Subtype.index_set())) ]

        return 'x'.join(Irreducible_Cartan_Strings)


    def Cartan_Type ( self ) :
        """Returns the Cartan type of the parabolic subgroup P."""
        return CartanType( self.Cartan_String() )


    def Connected_Components_Of_Included_Nodes ( self ) :
        """
        Returns the included nodes subdivided into lists of connected components
        (i.e. lists of nodes which are connected by edges).
        """
        return ( list( Dict.keys() ) for Dict in self.Relabelling(Decomposition_Style='Fine') )


    def Dimension ( self ) :
        """
        Returns the dimension of P.

        INPUT:
        - ``self`` -- ParabolicSubgroup; the parabolic subgroup P of a Cartan group G.

        OUTPUT:
        - ``Output`` -- Integer; the dimension of P.

        ALGORITHM:
        Thanks to the post by Pieter Belmans concerning the dimension of partial flag varieties
        from Jun 14th, 2017 on his blog (cf. to [Blog_PieterBelmans]_). The link is
        https://pbelmans.ncag.info/blog/2017/06/14/dimensions-of-partial-flag-varieties/
        (Date: Apr 21st, 2021).

        For the Borel group B ⊂ G: dim B = # of positive roots + rank (which accounts for the center)
        For P: dim P = dim B + # of negative roots which are added to construct P

        REFERENCE:
        [Blog_PieterBelmans] https://pbelmans.ncag.info/blog/
        """
        # Get the ingredients for the dimensions.
        Rank = self.Parent_Group().Cartan_Type().dynkin_diagram().rank()
        Number_Of_PositiveRoots = len( list( self.Parent_Group().Cartan_Type().root_system().root_lattice().positive_roots() ) )
        Number_Of_Available_NegativeRoots = self.Number_Of_NegativeRoots()
        # Assemble the dimensions out of the ingredients.
        return Rank + Number_Of_PositiveRoots + Number_Of_Available_NegativeRoots


    def Excluded_Nodes ( self , Output_Type=set ) :
        """Returns the excluded nodes."""
        Nodes = set( self.Parent_Group().Cartan_Type().index_set() )
        Output = list(Nodes.difference(self.Included_Nodes()))
        Output.sort()
        if Output_Type != set :
            if   Output_Type == list : return Output
            elif Output_Type == str : return ','.join(Output)
            else : raise ValueError('The output type is unknown.')
        else :
            return set(Output)


    def Included_Nodes ( self , Output_Type=set ) :
        """Returns the attribute ``_Included_Nodes``."""
        Output = list(self._Included_Nodes)
        Output.sort()
        if Output_Type != set :
            if   Output_Type == list : return Output
            elif Output_Type == str : return ','.join(Output)
            else : raise ValueError('The output type is unknown.')
        else :
            return set(Output)


    def Initialize_As_Cartan_Group ( self ) :
        """
        Returns the Cartan group associated to the Cartan string of P.
        Thus, one has access to the methods of the class ``Cartan_Group``.
        """
        return Cartan_Group( self.Cartan_String() )


    def Is_Minimal ( self ) :
        """Test if P is minimal (i.e. no node is included; or equivalently, all nodes are excluded)."""
        return self.Number_Of_Included_Nodes() == 0
    # Synonym for the function ``Is_Minimal``
    def Is_Borel ( self ) :
        """Test if P is Borel (i.e. it is minimal)."""
        return self.Is_Minimal()


    def Is_Complete ( self ) :
        """Test if P is complete (i.e. all nodes are included; or equivalently, no node is excluded)."""
        return self.Number_Of_Excluded_Nodes() == 0
    # Synonym for the function ``Is_Complete``
    def Is_Parent_Group ( self ) :
        """Test if P coincides with the parent group (i.e. it is complete)."""
        return self.Is_Complete()


    def Is_Maximal ( self ) :
        """Test if P is maximal (i.e. almost all nodes are included; or equivalently, there is only one node excluded)."""
        return self.Number_Of_Excluded_Nodes() == 1


    def Negative_Roots ( self ) :
        """Returns a list of all negative roots which are available for P."""
        return [ NegativeRoot for NegativeRoot in list( self.Parent_Group().Cartan_Type().root_system().root_lattice().negative_roots() )
                 if not False in [ Node in self.Included_Nodes() for Node , Coefficient in NegativeRoot ] ]


    def Number_Of_Excluded_Nodes ( self ) :
        """Returns the number of excluded nodes."""
        return len( self.Excluded_Nodes() )


    def Number_Of_Included_Nodes ( self ) :
        """Returns the number of included nodes."""
        return len( self.Included_Nodes() )


    def Number_Of_Negative_Roots ( self ) :
        """Returns the number of all negative roots which are available for P."""
        return len( self.NegativeRoots() )


    def Parent_Group ( self ) :
        """Returns the attribute ``_Parent_Group``."""
        return self._Parent_Group


    def Relabelling ( self , Decomposition_Style='Coarse' ) :
        """Returns the relabelling of the included nodes with respect to the parent group."""
        if   Decomposition_Style == 'Coarse' :
            return { Included_Node : Counter for Counter , Included_Node in enumerate( self.Included_Nodes(Output_Type=list) , start=1 ) }

        elif Decomposition_Style == 'Fine' :
            Connected_Components = []
            Included_Nodes = self.Included_Nodes(Output_Type=list)
            Cartan_Family = self.Parent_Group().Cartan_Family()
            Cartan_Degree = self.Parent_Group().Cartan_Degree()
            End = Cartan_Degree

            # For the case "Cartan_Family == E", the Dynkin-diagram-ordering is sligthly is different from a lexicographical-ordering.
            # In particluar, th second node is onyl one-connected to the fourth one and NOT between the third and fourth one.
            if Cartan_Family in [ 'A' , 'B' , 'C' , 'D' , 'F' , 'G' ] :
                Included_Nodes_In_Order_As_Given_By_DynkinDiagram = Included_Nodes
            elif Cartan_Family in [ 'E' ] :
                Included_Nodes_In_Order_As_Given_By_DynkinDiagram = [ Included_Node for Included_Node in [ 1 , 3 , 4 , 2 ] + [ 5 .. End ] if Included_Node in Included_Nodes ]

            # We establish connected components of all included nodes
            for Node_Counter , Current_Node in enumerate( Included_Nodes_In_Order_As_Given_By_DynkinDiagram , start=1 ) :
                Is_Current_Node_In_Some_Connected_Component = False

                # The first included nodes gives arise to the first connected component ...
                if Node_Counter == 1 :
                    Connected_Components += [ [ Current_Node ] ]
                    Is_Current_Node_In_Some_Connected_Component = True

                # Is the currently considered node element of an already existing connected component?
                else :
                    if Cartan_Family in [ 'A' , 'B' , 'C' , 'F' , 'G' ] :
                        for Connected_Component in Connected_Components :
                            if Current_Node-1 in Connected_Component :
                                Connected_Component += [ Current_Node ]
                                Is_Current_Node_In_Some_Connected_Component = True
                                break
                    elif Cartan_Family in [ 'D' ] :
                        for Connected_Component in Connected_Components :
                            if ( ( Current_Node <= End-2 and Current_Node-1 in Connected_Component ) or # k-th node is connected with the (k-1)-th node (k <= n-2)
                                 ( Current_Node == End-1 and End-1 in Connected_Component          ) or # (n-1)-th node is connected with the (n-2)-th node
                                 ( Current_Node == End   and End-2 in Connected_Component          )    # n-th node is connected with the (n-2)-th node
                               ) :
                                Connected_Component += [ Current_Node ]
                                Is_Current_Node_In_Some_Connected_Component = True
                                break
                    elif Cartan_Family in [ 'E' ] :
                        for Connected_Component in Connected_Components :
                            if ( ( Current_Node == 3 and Current_Node-2 in Connected_Component ) or # 3rd node
                                 ( Current_Node == 4 and Current_Node-1 in Connected_Component ) or # 4th node
                                 ( Current_Node == 2 and Current_Node+2 in Connected_Component ) or # 2nd node
                                 ( 5 <= Current_Node and Current_Node-1 in Connected_Component )    # 5th/ 6th/ ... node
                               ) :
                                Connected_Component += [ Current_Node ]
                                Is_Current_Node_In_Some_Connected_Component = True
                                break

                # If the currently considered node seems to be not element of one of the existing connected components.
                # Hence, we establish a new connected component ...
                if not Is_Current_Node_In_Some_Connected_Component : Connected_Components += [ [ Current_Node ] ]

            Relabelling = []
            for Connected_Component in Connected_Components :
                Connected_Component.sort()
                Relabelling += [ { Included_Node : Node_Counter_In_Connected_Component for Node_Counter_In_Connected_Component , Included_Node in enumerate( Connected_Component , start=1 ) } ]
            return Relabelling

        else : raise ValueError('The input for the argument of decomposition style is unknown.')